In [40]:
import requests
from bs4 import BeautifulSoup
from lxml import etree

from concurrent.futures import ThreadPoolExecutor
import pandas as pd
from datetime import datetime
from tqdm import tqdm
import os,json,time,random,warnings
warnings.filterwarnings('ignore')

In [41]:
def get_page_html(url, ret_type="text", timeout=5, encoding="utf-8"):
    """
    获取 html
    """
    headers = {
        # 'Host':'',
        # 'Referer':'',
        # 'Cookie':'',
        "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36"
    }
    res = requests.get(url=url,headers=headers,verify=False,timeout=timeout)
    time.sleep(random.random())
    res.encoding = encoding
    if ret_type == "text":
        return res.text
    elif ret_type == "image":
        return res.content
# html = get_page_html(url)
# soup = BeautifulSoup(html,"lxml")
# tree = etree.HTML(content) # 如果是response  content 替换为response.text
# # 查询id中包含‘l’的li标签
# li = tree.xpath('//ul/li[contains(@id,"l")]/text()')

In [42]:
os.makedirs('data',exist_ok=True)

#  爬取水稻数据

水稻：https://www.ricedata.cn/variety/index.htm 爬取到现在的全部数据

In [43]:
# 获取所有的省 和 初始地址
html = """
<div style="padding: 0 10px 0 10px; text-align:justify;text-justify:distribute-all-lines;text-align-last:justify; font-size:10pt; line-height:25px; font-weight:bold">
    <a
            href="identified/vejd_1.htm" target="_blank">浙江</a> ￤
    <a href="identified/fujm_1.htm" target="_blank">福建</a> ￤
    <a href="identified/jdxi_1.htm" target="_blank">江西</a><br>
    <a href="identified/jdsu_1.htm" target="_blank"><font
            color="#FF0000">江　苏</font></a> ￤
    <a href="identified/anhv_1.htm" target="_blank">安徽</a> ￤
    <a
            href="identified/uhhl_1.htm" target="_blank">上海</a> ￤
    <a href="identified/ujds_1.htm" target="_blank">山东</a><br>
    <a
            href="identified/hlj_1.htm" target="_blank"><font color="#FF0000">黑龙江</font></a> ￤
    <a
            href="identified/jxln_1.htm" target="_blank">吉林</a> ￤
    <a href="identified/lcny_1.htm" target="_blank">辽宁</a> ￤
    <a href="identified/hebz_1.htm" target="_blank">河北</a><br>
    <a href="identified/nmg_1.htm" target="_blank">内蒙古</a> ￤
    <a href="identified/ujxi_1.htm" target="_blank">山西</a> ￤
    <a href="identified/tmjn_1.htm" target="_blank">天津</a> ￤
    <a
            href="identified/bzjy_1.htm" target="_blank" style="color:gray">北京</a><br>
    <a href="identified/sjxi_1.htm"
       target="_blank">陕　西</a> ￤
    <a
            href="identified/gjsu_1.htm" target="_blank">甘肃</a> ￤
    <a href="identified/nyxw_1.htm" target="_blank">宁夏</a> ￤
    <a href="identified/xnjd_1.htm" target="_blank">新疆</a><br>
    <a href="identified/hunj_1.htm" target="_blank"><font
            color="#FF0000">湖　南</font></a> ￤
    <a href="identified/henj_1.htm" target="_blank">河南</a> ￤
    <a
            href="identified/hubz_1.htm" target="_blank">湖北</a> ￤
    <a
        href="identified/gdds_1.htm" target="_blank">广　东</a> ￤
    <a href="identified/gdxi_1.htm" target="_blank">广西</a> ￤
    <a href="identified/hlnj_1.htm" target="_blank">海南</a> ￤

    <a
        href="identified/siir_1.htm" target="_blank"><font color="#FF0000">四　川</font></a> ￤
    <a
            href="identified/ypnj_1.htm" target="_blank">云南</a>
    <a
        href="identified/gvvb_1.htm" target="_blank">贵州</a> ￤
    <a href="identified/isqy_1.htm" target="_blank">重庆</a><br>
</div>
"""
tree = etree.HTML(html) 
urls = tree.xpath('//a[contains(@href,"identified")]/@href')
urls = ['https://www.ricedata.cn/variety/'+i for i in urls]

names = []
for li in tree.xpath('//a[contains(@href,"identified")]'):
    name = li.xpath('./text()')
    if len(name) == 0:
        name = li.xpath('./font/text()')[0]
    else:
        name = name[0]
    names.append(name)

In [44]:
#  获取每个省份的数据
datas = []


import threading
import time

class MyThread(threading.Thread):
    def __init__(self, func, args, name=''):
        super().__init__(name=name)
        self.func = func
        self.args = args
    def run(self):
        print(f"线程 {self.name} 启动")
        self.func(*self.args)
        print(f"线程 {self.name} 结束")

# 示例用法
def example_function(url, name):
    # 执行一些操作
#     print(f"线程参数 {url} and {name}")
    html = get_page_html(url,encoding='gb2312')
    tree = etree.HTML(html) # 如果是response  content 替换为response.text
    trs = tree.xpath('//tr')
    # 如果没找到 既要退出了
    if len(trs)==0:
        return
    for tr in trs[2:-1]:
        # 原产地
        try:
            pos = tr.xpath('./td[5]/text()')[0]
        except:
            pos = ''
        one = [
            name,
            tr.xpath('./td[2]/a/text()')[0],
            tr.xpath('./td[3]/text()')[0],
            tr.xpath('./td[4]/text()')[0],
            pos,
            tr.xpath('./td[6]/text()')[0],
        ]
        datas.append(one)

def main():
    threads = []
    for name,first_url in zip(names,urls):
        # 最大页
        html = get_page_html(first_url,encoding='gb2312')
        tree = etree.HTML(html) # 如果是response  content 替换为response.text
        max_page = tree.xpath('//b/a/@href')[-1].split('.')[0].split('_')[-1]
        max_page = int(max_page)
        for page in range(1,max_page + 1):
            url = first_url[:-5] + str(page) + first_url[-4:]
            # 创建线程实例
            thread = MyThread(func=example_function, args=(url, name), name=f"Thread-{name}-{page}/{max_page}")
            threads.append(thread)

    # 启动线程
    for t in threads:
        t.start()
    # 等待线程结束
    for t in threads:
        t.join()

    print("所有线程结束")

if __name__ == '__main__':
    main()
# 保存数据
df1 = pd.DataFrame(data=datas, columns=['省份','品种名称','亲本来源','类型','原产地','审定编号'])
df1.to_csv('data/水稻数据.csv',index=None)

线程 Thread-浙江-1/22 启动
线程 Thread-浙江-2/22 启动
线程 Thread-浙江-3/22 启动
线程 Thread-浙江-4/22 启动
线程 Thread-浙江-5/22 启动
线程 Thread-浙江-6/22 启动
线程 Thread-浙江-7/22 启动
线程 Thread-浙江-8/22 启动
线程 Thread-浙江-9/22 启动
线程 Thread-浙江-10/22 启动
线程 Thread-浙江-11/22 启动
线程 Thread-浙江-12/22 启动
线程 Thread-浙江-13/22 启动
线程 Thread-浙江-14/22 启动
线程 Thread-浙江-15/22 启动
线程 Thread-浙江-16/22 启动
线程 Thread-浙江-17/22 启动
线程 Thread-浙江-18/22 启动
线程 Thread-浙江-19/22 启动
线程 Thread-浙江-20/22 启动
线程 Thread-浙江-21/22 启动
线程 Thread-浙江-22/22 启动
线程 Thread-福建-1/26 启动
线程 Thread-福建-2/26 启动
线程 Thread-福建-3/26 启动
线程 Thread-福建-4/26 启动
线程 Thread-福建-5/26 启动
线程 Thread-福建-6/26 启动
线程 Thread-福建-7/26 启动
线程 Thread-福建-8/26 启动
线程 Thread-福建-9/26 启动
线程 Thread-福建-10/26 启动
线程 Thread-福建-11/26 启动
线程 Thread-福建-12/26 启动
线程 Thread-福建-13/26 启动
线程 Thread-福建-14/26 启动
线程 Thread-福建-15/26 启动
线程 Thread-福建-16/26 启动
线程 Thread-福建-17/26 启动
线程 Thread-福建-18/26 启动
线程 Thread-福建-19/26 启动
线程 Thread-福建-20/26 启动
线程 Thread-福建-21/26 启动
线程 Thread-福建-22/26 启动
线程 Thread-福建-23/26 启动
线程 Thread-福建-24/26 启动
线程 Thr

# 蔬菜爬取

蔬菜等农产品：http://www.wbncp.com/?m=home&c=Lists&a=index&tid=69 爬取到现在的全部数据

In [45]:
# 1708963200000 2月 27 
# 1706716800000 2月 1
def time_str_to_timestamp(date_string = '2025-03-20',length=10):
    """
    date_string 日期字符
    length 转换为多少位
    时间转化为时间戳 10位数 
    date_string = '2024-02-01'
    time_str_to_timestamp(date_string)
    # 1706716800
    """
    import datetime
    import time
    format_string = "%Y-%m-%d"
    timestamp = int(time.mktime(datetime.datetime.strptime(date_string, format_string).timetuple()))
    if length==13:
        timestamp = timestamp*1000
    
    return timestamp
date_string = '2025-03-20'
time_str_to_timestamp(date_string,length=13)
# 1706716800
def post_get_html(url,payload):
    headers = {
        "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36",
        'Accept':'application/json, text/plain, */*',
        'Content-Type':'application/json',
    }
    
    res = requests.post(url=url,data=json.dumps(payload),headers=headers,verify=False)
    time.sleep(random.random())
    time.sleep(10)
    
    return res.json()
# 获取过去多少天的日期列表 
def get_past_date(end_time_str='2025-03-22',days=30,is_str=True):
    """
    获取过去多少天的日期列表 
    end_time_str 起止时间 
    days 过去多少天
    is_str 是否字符串
    get_past_date(end_time_str='2024-02-27',days=30,is_str=True)
    """
    import datetime
    this_time = datetime.datetime.strptime(end_time_str, '%Y-%m-%d') # 日期对象
    data = []
    for i in range(days):
        tmp_time = this_time-datetime.timedelta(days=i)
        data.append(tmp_time)
    # 转换字符串
    if is_str:
        data = [datetime.datetime.strftime(i, '%Y-%m-%d') for i in data] # 日期字符串
    return data


In [ ]:
import threading
import time

df2 = pd.DataFrame()

class MyThread(threading.Thread):
    def __init__(self, func, args, name=''):
        super().__init__(name=name)
        self.func = func
        self.args = args
    def run(self):
        # print(f"线程 {self.name} 启动")
        self.func(*self.args)
        print(f"线程 {self.name} 结束")

# 示例用法
def example_function(kind, date_str):
    
    # 执行一些操作
#     print(f"线程参数 {arg1} and {arg2}")
    time.sleep(5)
    global df2
    try:
        timestamp = time_str_to_timestamp(date_string = date_str,length=13)
        url = "https://wap.api.banglail.com:62021/api-entrance/productcategory/officialWebsite"
        payload = {"page":1,"limit":1000,"categoryName":kind,"createTime":timestamp}
        data2  = post_get_html(url,payload=payload)
        data2 = data2['data']['list']
        
        df_tmp = pd.DataFrame(data2)
        # 分类 品名 最高价 平均价  最低价 单位 规格 产地 发布日期 
        columns = ['categoryName', 'productName', 'priceMax', 'priceAvg','priceMin',
                   'unit', 'productSize',  'productPlace', 'recordDate', ]
        df_tmp = df_tmp[columns]
        df2 = pd.concat([df2,df_tmp])
    
        print(f"线程:{kind}{date_str}结束")
    except Exception as e:
        print(f"线程:{kind}{date_str}异常，{e}")

def main():
    threads = []
    for kind in ['蔬菜类','果品类','水产','畜禽蛋品','粮油','副食']:
        # 30tian
        for date_str in get_past_date(end_time_str='2025-03-22',days=30,is_str=True):
        
            # 创建线程实例
            thread = MyThread(func=example_function, args=(kind, date_str), name=f"Thread-{kind}-{date_str}")
            threads.append(thread)

    # 启动线程
    for t in threads:
        t.start()
    # 等待线程结束
    for t in threads:
        t.join()

    print("所有线程结束")
if __name__ == '__main__':
    main()
df2.to_csv('data/农产品数据.csv',index=None)

线程:水产2025-03-11异常，"None of [Index(['categoryName', 'productName', 'priceMax', 'priceAvg', 'priceMin',\n       'unit', 'productSize', 'productPlace', 'recordDate'],\n      dtype='object')] are in the [columns]"
线程 Thread-水产-2025-03-11 结束
线程:水产2025-03-03异常，"None of [Index(['categoryName', 'productName', 'priceMax', 'priceAvg', 'priceMin',\n       'unit', 'productSize', 'productPlace', 'recordDate'],\n      dtype='object')] are in the [columns]"
线程 Thread-水产-2025-03-03 结束
线程:水产2025-03-01异常，"None of [Index(['categoryName', 'productName', 'priceMax', 'priceAvg', 'priceMin',\n       'unit', 'productSize', 'productPlace', 'recordDate'],\n      dtype='object')] are in the [columns]"线程:水产2025-02-21异常，"None of [Index(['categoryName', 'productName', 'priceMax', 'priceAvg', 'priceMin',\n       'unit', 'productSize', 'productPlace', 'recordDate'],\n      dtype='object')] are in the [columns]"
线程 Thread-水产-2025-02-21 结束
线程:水产2025-03-13异常，"None of [Index(['categoryName', 'productName', 'priceMax', 'p